<a href="https://colab.research.google.com/github/Puisque/object-detection/blob/main/Step_1_Get_youtube_ids.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**The below codes are for Video Database Module.**

#### **Main Notes**:

Logic:
1. Get **TMDB ids** from TMDB dataset
2. Use **API key & TMDB ids** to get **YouTube ids** from TMDB API
3. Use YouTube ids to format YouTube links, which will be used by Pafy.

---
Instruction: How to get movie trailer YouTube ids from TMDb API: https://oupton.github.io/startup-ds/. Note:*The TMDB API key can be found in our Code folder*.

TMDB dataset (where we get tmdbid): https://www.kaggle.com/rounakbanik/the-movies-dataset?select=keywords.csv. 

Note: 
1. The dataset consists of movies released on or before July 2017.
2. The dataset has been uploaded to our Code folder.



In [ ]:
import time
import requests
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


#### **Get TMDB ids from the TMDB dataset**

In [ ]:
def read_dataset_csv(fileName):
  file = pd.read_csv(
    "/content/gdrive/MyDrive/Capstone/Code/TMDB/"+fileName,
    usecols = header,
    index_col=False)

  return file

In [ ]:
fileName = 'tmdb_movies_metadata.csv' # can be found in the Code -> TMDB folder 
header = ['genres','homepage','id','original_language','original_title','overview', 'poster_path','release_date', 'runtime', 'title',]#10 columns 
fileList_1 = read_dataset_csv(fileName)
print(fileList_1.shape)

(45466, 10)


In [ ]:
#For test: use the first 100 movies
test_list = fileList_1[:100].to_dict('index')

#### **Get YouTube ids from TMDB API**

In [ ]:
TMDB_API_KEY = 'e0476a281215758d95503294fc0cb530' # can be found in the Code -> TMDB folder 
TMDB_VIDEO_URL = 'https://api.themoviedb.org/3/movie/{}/videos?api_key={}'

Pull the YouTube trailer IDs of the movies using the TMDb API.

Please note that TMDb has a ***40 request per 10 second interval rate limiting***, so be cautious not to upset this.

In [ ]:
def get_youtube_data(video_data):
  trailers = {}
  url_base = 'https://www.youtube.com/watch?v='
  for i,(key, value) in enumerate(video_data.items()):
      try:  
        url = TMDB_VIDEO_URL.format(value['id'], TMDB_API_KEY)
        resp = requests.get(url)
        if resp.status_code != 200:
            print('Error: Failed to retrieve video for TMDb ID {} with status code: {}'\
                    .format(value['id'], resp.status_code))
            trailers.append([])
        else:
            youtube_results = [] #a list of ids
            for video in resp.json()['results']:
              if video['site'] == 'YouTube' and (video['type'] in { 'Teaser', 'Trailer' }) and video['key'] != '':
                  youtube_results.append([video['name'],url_base+video['key']])
            if len(youtube_results) >0:
              temp = {}
              temp['Youtube Info'] = youtube_results
              temp['Video Info'] = value
              trailers[value['id']] = temp

        # Rate limiting, stall 10 seconds
        if i != 0 and i % 40 == 0:
            print('Sleep')
            time.sleep(10)
      except:
        print('Send Request Failed')
  
  return trailers

In [ ]:
test_trailer_list = get_youtube_data(test_list)

Sleep
Sleep


In [ ]:
#Visualize Data
n = 0
for key, value in test_trailer_list.items():
  print('TMDB ID = ',key)
  print('Youtube Info = ',value['Youtube Info'])
  print('Movie Info = ',value['Video Info'])
  print('--------------------------------------------')
  n+=1
  if (n>3): 
    break


TMDB ID =  862
Youtube Info =  [['Toy Story Teaser Trailer HD Widescreen', 'https://www.youtube.com/watch?v=4j_qfJN0qd4'], ['Toy Story Trailer 1995 | Disney Throwback | Oh My Disney', 'https://www.youtube.com/watch?v=tN1A2mVnrOM'], ['Toy Story (1995) Trailer #1 | Movieclips Classic Trailers', 'https://www.youtube.com/watch?v=v-PjgYDrg70']]
Movie Info =  {'genres': "[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]", 'homepage': 'http://toystory.disney.com/toy-story', 'id': '862', 'original_language': 'en', 'original_title': 'Toy Story', 'overview': "Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.", 'poster_path': '/rhIRbceoE9lR4veEXuwCC2wARtG.jpg', 'release_date': '1995-10-30', 'runtime':

#### **Apply Filter**

In [ ]:
!pip install pafy
!pip install youtube_dl

In [ ]:
!pip install tensorflow==1.13.1
!pip install keras==2.3.0

In [ ]:
import pafy
import cv2
import ast

In [ ]:
def genres_filter(data,genres):
  genres_list = ast.literal_eval(data['Video Info']['genres'])
  found = False
  for each in genres_list:
    if each['name'] == genres:
      found = True
      print('Genres Match')
      break;   
  return found

In [ ]:
def length_filter(video_streams,length):
  fps = video_streams.get(cv2.CAP_PROP_FPS) 
  frame_count = int(video_streams.get(cv2.CAP_PROP_FRAME_COUNT))
  video_length = frame_count/fps
  print('Duration: {} sec '.format(video_length) )
  return True if video_length<length else False

In [ ]:
def get_video_data(num_videos, length,genres,video_list):

  video_list_copy = dict(video_list)
  n = 0
  for i,(key, value) in enumerate(video_list_copy.items()):
      print('------------------ ID {} ------------------'.format(key))
      streams = [];

      #Filter: Genres
      if genres  != 'NA':
        temp = genres_filter(value,genres)
        if (temp == False): 
          continue
      for eachVideo in value['Youtube Info']:
        url = eachVideo[1]
        try:
          n+=1
          youtube_video = pafy.new(url)
          youtube_video = youtube_video.getbest()
          video_streams = cv2.VideoCapture(youtube_video.url)
          #Filter: Video Length
          if (length_filter(video_streams,length)):
            print('Within Limit')
            streams.append([eachVideo[0],eachVideo[1],video_streams])
        except:
          print('Pafy Get Wrong')
        
        if n != 0 and n % 20 == 0:
            print('===================== Sleep =====================')
            time.sleep(10)

      video_list_copy[key]['Youtube Info'] = streams
      if i>=num_videos:
        break;
  
  return video_list_copy


In [ ]:
num_videos=10
length = 120 #max length , sec
genres = 'Comedy'
video_data_stream = get_video_data(num_videos,length,genres,test_trailer_list)

------------------ ID 862 ------------------
Genres Match
------------------ ID 8844 ------------------
------------------ ID 15602 ------------------
Genres Match
------------------ ID 31357 ------------------
Genres Match
------------------ ID 11862 ------------------
Genres Match
------------------ ID 949 ------------------
------------------ ID 11860 ------------------
Genres Match
------------------ ID 45325 ------------------
------------------ ID 9091 ------------------
------------------ ID 710 ------------------
------------------ ID 9087 ------------------
Genres Match


In [ ]:
#Demo
for key, value in test_trailer_list.items():
  for each in value:

    url = each[1]
    try:
      vPafy = pafy.new(url)
      play = vPafy.getbest()
      video_streams = cv2.VideoCapture(play.url)
      
      fps = video_streams.get(cv2.CAP_PROP_FPS) 
      frame_count = int(video_streams.get(cv2.CAP_PROP_FRAME_COUNT))
      duration = frame_count/fps
      print('Video Title : {} ------- Duration: {} sec '.format(each[0],duration) )
    except:
      print('Pafy Get Wrong')


Video Title : Toy Story Teaser Trailer HD Widescreen ------- Duration: 89.65623333333333 sec 
Video Title : Toy Story Trailer 1995 | Disney Throwback | Oh My Disney ------- Duration: 187.10357777777776 sec 
Video Title : Toy Story (1995) Trailer #1 | Movieclips Classic Trailers ------- Duration: 88.04628888888888 sec 
Video Title : Jumanji Trailer 1995 ------- Duration: 167.20870833333333 sec 
Video Title : Jumanji (1995) International Teaser ------- Duration: 105.14670833333332 sec 
Video Title : Jumanji (1995) Trailer #1 |  Movieclips Classic Trailers ------- Duration: 171.171 sec 
Video Title : Grumpier Old Men - Trailer ------- Duration: 111.8117 sec 
Video Title : Waiting To Exhale ------- Duration: 159.32583333333332 sec 
Video Title : Father of the Bride Part II (1995) Trailer ------- Duration: 155.6555 sec 
Video Title : Heat - Original Theatrical Trailer ------- Duration: 135.875 sec 
Video Title : Sabrina (1995) - Trailer ------- Duration: 169.41924999999998 sec 
Video Title 

ERROR: Video unavailable


Video Title : Sudden Death Trailer 1995 ------- Duration: 30.029999999999998 sec 
Pafy Get Wrong
Video Title : Goldeneye - Teaser Trailer ------- Duration: 109.83333333333333 sec 
Video Title : 007: GoldenEye (1995) Trailer - Secret Weapon ------- Duration: 171.00416666666666 sec 
Video Title : 007: GoldenEye (1995) Trailer - New World Bond ------- Duration: 96.471375 sec 
Video Title : The American President 1995 Movie Trailer ------- Duration: 169.169 sec 
Video Title : Dracula: Dead and Loving It - Trailer #1 ------- Duration: 119.29166666666667 sec 
Video Title : Balto Trailer ------- Duration: 132.36 sec 
Video Title : NIXON Original Theatrical Trailer ------- Duration: 272.93933333333337 sec 
Video Title : Cutthroat Island - Trailer ------- Duration: 104.97987499999999 sec 
Video Title : Casino Official Trailer #1 - (1995) HD ------- Duration: 182.68249999999998 sec 
Video Title : Sense and Sensibility (1995) Trailer ------- Duration: 135.135 sec 


ERROR: Video unavailable


Video Title : Sense and Sensibility (1995) Trailer #1 | Movieclips Classic Trailers ------- Duration: 136.84504166666665 sec 
Pafy Get Wrong
Video Title : Four Rooms - Trailer ------- Duration: 127.33554166666666 sec 
Video Title : Ace Ventura: When Nature Calls ------- Duration: 118.3182 sec 
Video Title : Ace Ventura: When Nature Calls - Trailer ------- Duration: 117.75096666666667 sec 
Video Title : Money Train (Theatrical Trailer) ------- Duration: 162.496 sec 
Video Title : Get Shorty Official Trailer #1 - Gene Hackman Movie (1995) HD ------- Duration: 154.6962 sec 
Video Title : Copycat (1995) trailer ------- Duration: 120.07829166666666 sec 
Video Title : Assassins (1995) ------- Duration: 159.8597 sec 
Video Title : Powder Official 1995 Movie Trailer ------- Duration: 154.33333333333331 sec 
Video Title : LEAVING LAS VEGAS - Trailer ( 1995 ) ------- Duration: 120.5204 sec 
Video Title : Now and Then (1995) Official Trailer 1 - Christina Ricci, Rosie O'Donnell Movie HD ------- D

...

Pafy Get Wrong
Video Title : The Big Green 1995 Trailer ------- Duration: 136.04 sec 
Video Title : The Big Green (1995) Trailer ------- Duration: 140.18170833333332 sec 
Video Title : Georgia - Trailer ------- Duration: 96.59649999999999 sec 
Video Title : Home For The Holidays Trailer 1995 ------- Duration: 182.34883333333332 sec 
Video Title : IL POSTINO TRAILER ------- Duration: 113.113 sec 
Video Title : The Indian In The Cupboard Trailer 1995 ------- Duration: 121.6215 sec 
Video Title : Eye for an Eye (1996) Trailer ------- Duration: 55.0 sec 
Video Title : Richard Dreyfuss in "Mr Hollands Opus" 1996 Movie Trailer ------- Duration: 63.07520891364902 sec 
Video Title : Don't Be a Menace to South Central While Drinking Your Juice in the Hood (1996) Official Trailer #1 ------- Duration: 91.00757777777778 sec 
Video Title : Two If By Sea ------- Duration: 116.08263333333333 sec 
Video Title : Bio-Dome Official Trailer #1 - Pauly Shore Movie (1996) HD ------- Duration: 152.5690833333

ERROR: Video unavailable


Video Title : Two Bits - Trailer ------- Duration: 138.09629166666664 sec 
Pafy Get Wrong
Video Title : Friday (1995) Official Trailer - Ice Cube, Chris Tucker Comedy HD ------- Duration: 163.42993333333334 sec 
Video Title : From Dusk Till Dawn Official Trailer #1 - (1996) HD ------- Duration: 121.5801 sec 
Video Title : Official Trailer: From Dusk Till Dawn (1996) ------- Duration: 86.80779944289694 sec 
Video Title : Fair Game Trailer 1995 ------- Duration: 112.77933333333333 sec 
Video Title : Kicking And Screaming - Trailer ------- Duration: 99.39095833333333 sec 
Video Title : Les Miserables Trailer 1995 ------- Duration: 105.14670833333332 sec 
Video Title : Bed of Roses (1996) Trailer ------- Duration: 124.13333333333334 sec 
Video Title : Big Bully - Trailer ------- Duration: 95.06163333333333 sec 
Video Title : Screamers Trailer 1996 ------- Duration: 116.24066666666667 sec 
Video Title : The Crossing Guard (1995) Official Trailer 1 - Jack Nicholson Movie ------- Duration: 11

ERROR: This video contains content from AdRevEnt_managed, who has blocked it on copyright grounds.


Pafy Get Wrong
Video Title : Heidi Fleiss: Hollywood Madam ------- Duration: 67.8 sec 
Video Title : City Hall - Trailer ------- Duration: 124.0239 sec 
Pafy Get Wrong
Video Title : Bottle Rocket (1996) Trailer #1 | Movieclips Classic Trailers ------- Duration: 173.04787499999998 sec 


In [ ]:
#To Be Implemented